In [ ]:
Rachel Cu

# Gradient Descent and Artificial Neural Networks

### Problem: Use the titanic train.csv file and code a neural network from scratch to predict if a person survived based on three input variables x1, x2, and x3 (for example, Sex, Age and Fare).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

### Import the data and drop the columns that are not important for this analysis.

In [2]:
data = pd.read_csv('titanic_train.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.drop(['PassengerId', 'Pclass', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked'], axis = 1, inplace = True)

In [5]:
data.head()

,Survived,Sex,Age,Fare
0,0,male,22.0,7.2500
1,1,female,38.0,71.2833
2,1,female,26.0,7.9250
3,1,female,35.0,53.1000
4,0,male,35.0,8.0500


### Inspect the data for the missing values. 

In [6]:
data.isnull().sum()

Survived      0
Sex           0
Age         177
Fare          0
dtype: int64

### Fill missing values in Age column using median

In [7]:
median_age = data['Age'].median()
median_age

28.0

In [8]:
data['Age'].fillna(median_age, inplace=True)

In [9]:
data.isnull().sum()

Survived    0
Sex         0
Age         0
Fare        0
dtype: int64

### Convert Sex values into numbers

In [10]:
data["Sex"].replace('female', 0, inplace=True)
data['Sex'].replace('male', 1,inplace=True)

In [11]:
data.head()

,Survived,Sex,Age,Fare
0,0,1,22.0,7.2500
1,1,0,38.0,71.2833
2,1,0,26.0,7.9250
3,1,0,35.0,53.1000
4,0,1,35.0,8.0500


### Scale Age and Fare columns

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
scaler = MinMaxScaler()

In [14]:
data[['Age', 'Fare']]= scaler.fit_transform(data[['Age', 'Fare']])

In [15]:
data.head()

,Survived,Sex,Age,Fare
0,0,1,0.271174,0.014151
1,1,0,0.472229,0.139136
2,1,0,0.321438,0.015469
3,1,0,0.434531,0.103644
4,0,1,0.434531,0.015713


### Split the data into training and testing sets.


In [16]:
X = data[['Sex', 'Age', 'Fare']]
y = data['Survived']

In [17]:
X

,Sex,Age,Fare
0,1,0.271174,0.014151
1,0,0.472229,0.139136
2,0,0.321438,0.015469
3,0,0.434531,0.103644
4,1,0.434531,0.015713
...,...,...,...
886,1,0.334004,0.025374
887,0,0.233476,0.058556
888,0,0.346569,0.045771
889,1,0.321438,0.058556


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=45)

In [19]:
X_train.iloc[0].shape

(3,)

###  Build a simple neural network from scratch with three input nodes and one output node

In [20]:
import math

In [21]:
 def sigmoid_numpy(x):
    return 1/(1+ np.exp(-x))

In [22]:
def loss_function(y_true, y_predicted):
    epsilon = 0.00000000000001
    y_predicted_new = [max(y, epsilon) for y in y_predicted]
    y_predicted_new = [min(y, 1-epsilon) for y in y_predicted]
    y_predicted_new = np.array(y_predicted)
    return -np.mean(y_true * np.log(y_predicted_new) + (1-y_true) * np.log(1-y_predicted_new))

In [23]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.w3 = 1
        self.b = 0
        
    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.w2, self.w3, self.b = self.gradient_descent(X['Sex'], X['Age'], X['Fare'], y, epochs, loss_threshold)
        
    def predict(self, X):
        weighted_sum = self.w1 * X['Sex'] + self.w2 * X['Age'] + self.w3 * X['Fare'] + self.b
        y_predicted = sigmoid_numpy(weighted_sum)
        return y_predicted
    
    def gradient_descent(self, sex, age, fare, y_true, epochs, loss_threshold):
        w1 = 1
        w2 = 1
        w3 = 1
        b = 0
        learning_rate = 0.5
        n = len(age)
        
        for i in range(epochs):
            weighted_sum = w1 * sex + w2 * age + w3 * fare + b
            y_predicted = sigmoid_numpy(weighted_sum)
            
            loss = loss_function(y_true, y_predicted)
            
            w1_d = (1/n) * np.dot(sex, y_predicted - y_true)
            w2_d = (1/n) * np.dot(age, y_predicted - y_true)
            w3_d = (1/n) * np.dot(fare, y_predicted - y_true)
            b_d = np.mean(y_predicted - y_true)
            
            w1 = w1 - learning_rate * w1_d
            w2 = w2 - learning_rate * w2_d
            w3 = w3 - learning_rate * w3_d
            b = b - learning_rate * b_d
            
            print(f'Epoch: {i+1}, w1: {w1}, w2: {w2}, w3: {w3}, bias:{b}, loss: {loss}')
            
            if loss < loss_threshold:
                break
                
        return w1, w2, w3, b

In [24]:
scratch_model = myNN()
scratch_model.fit(X_train, y_train, 1000, 0.51)

Epoch: 1, w1: 0.8094506077346826, w2: 0.9324956629116281, w3: 0.9955389566048864, bias:-0.1650672262140765, loss: 1.0773497796966287
Epoch: 2, w1: 0.6403108764555319, w2: 0.8756282463757742, w3: 0.9927967275763201, bias:-0.3007245524187672, loss: 0.9504540645956774
Epoch: 3, w1: 0.4928631193278467, w2: 0.8291678131154406, w3: 0.9917055667444425, bias:-0.407946431141632, loss: 0.8579108731605325
Epoch: 4, w1: 0.36549057254169287, w2: 0.7921213165461127, w3: 0.992090786506443, bias:-0.4897016911695306, loss: 0.7931755806386545
Epoch: 5, w1: 0.2555146311321695, w2: 0.7630967380451104, w3: 0.9937243594454235, bias:-0.5499311866632984, loss: 0.7488632561534064
Epoch: 6, w1: 0.16000459293138292, w2: 0.7406422952789964, w3: 0.996373370807918, bias:-0.5926444892431924, loss: 0.7185579691053476
Epoch: 7, w1: 0.07626131191747981, w2: 0.7234445966250262, w3: 0.999828111245624, bias:-0.6214165614250001, loss: 0.6974780739344998
Epoch: 8, w1: 0.002003776831434795, w2: 0.7104010241100751, w3: 1.0039

Epoch: 86, w1: -1.765911909748229, w2: 0.6534445306598922, w3: 1.3769443577695741, bias:0.31187627613755026, loss: 0.5382957496699731
Epoch: 87, w1: -1.7751832898382998, w2: 0.6515808437856161, w3: 1.3807297531543077, bias:0.31892800824642353, loss: 0.5379818816730998
Epoch: 88, w1: -1.7842933526113633, w2: 0.6496938984305264, w3: 1.3844993609432397, bias:0.3258762581179256, loss: 0.5376771482781114
Epoch: 89, w1: -1.7932451646401442, w2: 0.6477842706334176, w3: 1.3882534553374037, bias:0.33272294335041586, loss: 0.5373812281930004
Epoch: 90, w1: -1.802041727127987, w2: 0.6458525224983336, w3: 1.3919923039563824, bias:0.339469939259974, loss: 0.5370938122965183
Epoch: 91, w1: -1.8106859774158668, w2: 0.6438992026005472, w3: 1.3957161680249743, bias:0.3461190799820052, loss: 0.536814603147897
Epoch: 92, w1: -1.819180790456679, w2: 0.6419248463789916, w3: 1.3994253025540038, bias:0.3526721595410406, loss: 0.5365433145173015
Epoch: 93, w1: -1.8275289802571757, w2: 0.6399299765155774, w3: 

Epoch: 160, w1: -2.158891135659713, w2: 0.4786976999898225, w3: 1.6268099775094342, bias:0.6430278917843631, loss: 0.5276871293011461
Epoch: 161, w1: -2.1615632560449756, w2: 0.4760788876687956, w3: 1.6298878610784318, bias:0.6457479681197493, loss: 0.527624748692005
Epoch: 162, w1: -2.164191975135685, w2: 0.473457885126052, w3: 1.6329600245061366, bias:0.6484381604834503, loss: 0.5275632166786762
Epoch: 163, w1: -2.1667780064916373, w2: 0.4708348129906445, w3: 1.6360265265494045, bias:0.6510989215783247, loss: 0.527502509784741
Epoch: 164, w1: -2.169322051029118, w2: 0.4682097896226841, w3: 1.6390874249212188, bias:0.653730696324591, loss: 0.5274426052849099
Epoch: 165, w1: -2.1718247972724196, w2: 0.46558293115890625, w3: 1.6421427763126464, bias:0.6563339220115241, loss: 0.527383481179593
Epoch: 166, w1: -2.174286921599644, w2: 0.4629543515571436, w3: 1.6451926364142526, bias:0.6589090284457043, loss: 0.5273251161703925
Epoch: 167, w1: -2.1767090884829305, w2: 0.4603241626397381, w3

Epoch: 238, w1: -2.2773823434488394, w2: 0.27453893957108044, w3: 1.8530447728855304, bias:0.791298382895324, loss: 0.5242851458579337
Epoch: 239, w1: -2.2781172547521504, w2: 0.27198685645693127, w3: 1.85579640506111, bias:0.7926164132834532, loss: 0.5242523070311498
Epoch: 240, w1: -2.278839722695975, w2: 0.26943755644868395, w3: 1.8585448980221642, bias:0.7939245290431591, loss: 0.5242196211742679
Epoch: 241, w1: -2.2795499409323505, w2: 0.2668910666514417, w3: 1.8612902686852195, bias:0.7952228585010306, loss: 0.5241870860328527
Epoch: 242, w1: -2.2802481000626944, w2: 0.2643474136164184, w3: 1.8640325337287302, bias:0.7965115280397062, loss: 0.5241546994126104
Epoch: 243, w1: -2.280934387688468, w2: 0.2618066233500686, w3: 1.8667717095971352, bias:0.7977906621294466, loss: 0.5241224591775734
Epoch: 244, w1: -2.2816089884609156, w2: 0.2592687213230564, w3: 1.8695078125048372, bias:0.7990603833591479, loss: 0.5240903632483354
Epoch: 245, w1: -2.2822720841299065, w2: 0.25673373247906

Epoch: 326, w1: -2.310140837716187, w2: 0.06257929394463038, w3: 2.0845984516310954, bias:0.8798722805828239, loss: 0.5218247514148289
Epoch: 327, w1: -2.310277358931832, w2: 0.0603314680069788, w3: 2.0871196138757204, bias:0.8806522260369575, loss: 0.5218006273102107
Epoch: 328, w1: -2.3104107700592365, w2: 0.058087412737846976, w3: 2.0896385286473973, bias:0.8814286005317113, loss: 0.5217765727959044
Epoch: 329, w1: -2.3105411206083386, w2: 0.05584712920056153, w3: 2.09215520172805, bias:0.8822014395333666, loss: 0.5217525875173787
Epoch: 330, w1: -2.310668459337336, w2: 0.05361061832707421, w3: 2.0946696388419075, bias:0.8829707780160964, loss: 0.5217286711249295
Epoch: 331, w1: -2.3107928342641446, w2: 0.051377880920133824, w3: 2.0971818456563796, bias:0.8837366504693229, loss: 0.5217048232735525
Epoch: 332, w1: -2.3109142926776762, w2: 0.04914891765542506, w3: 2.0996918277829093, bias:0.8844990909049607, loss: 0.52168104362283
Epoch: 333, w1: -2.3110328811489373, w2: 0.04692372908

...

Epoch: 598, w1: -2.308107257819746, w2: -0.4205523824848611, w3: 2.699747589666378, bias:1.019196733886918, loss: 0.5171188889590782
Epoch: 599, w1: -2.3080694217864886, w2: -0.421914961767829, w3: 2.701779468588907, bias:1.019545555566825, loss: 0.5171066519806642
Epoch: 600, w1: -2.3080316383403194, w2: -0.4232749981783311, w3: 2.703809845421858, bias:1.0198935604357597, loss: 0.5170944422599907
Epoch: 601, w1: -2.3079939079454346, w2: -0.4246324967523161, w3: 2.7058387218184388, bias:1.0202407506722118, loss: 0.5170822597155551
Epoch: 602, w1: -2.307956231054559, w2: -0.42598746251734004, w3: 2.707866099428977, bias:1.0205871284429473, loss: 0.5170701042661442
Epoch: 603, w1: -2.307918608109134, w2: -0.4273399004925486, w3: 2.709891979900935, bias:1.0209326959031422, loss: 0.5170579758308269
Epoch: 604, w1: -2.307881039539503, w2: -0.4286898156886608, w3: 2.7119163648789217, bias:1.0212774551965098, loss: 0.517045874328964
Epoch: 605, w1: -2.3078435257650898, w2: -0.4300372131079528

Epoch: 690, w1: -2.3048802283712866, w2: -0.5358736612717989, w3: 2.8805928563738665, bias:1.0481153211242655, loss: 0.5160977063225698
Epoch: 691, w1: -2.304848142616953, w2: -0.5370222968883018, w3: 2.88249313412582, bias:1.048396935490596, loss: 0.5160876691758021
Epoch: 692, w1: -2.3048161214551683, w2: -0.5381688168530351, w3: 2.8843920512135512, bias:1.0486778968114139, loss: 0.5160776528668205
Epoch: 693, w1: -2.304784164816176, w2: -0.5393132254086331, w3: 2.886289609067899, bias:1.0489582065637861, loss: 0.5160676573364972
Epoch: 694, w1: -2.304752272628052, w2: -0.540455526789277, w3: 2.888185809117619, bias:1.0492378662199202, loss: 0.5160576825259013
Epoch: 695, w1: -2.3047204448167475, w2: -0.5415957252207033, w3: 2.8900806527893903, bias:1.0495168772472019, loss: 0.5160477283763076
Epoch: 696, w1: -2.3046886813061365, w2: -0.5427338249202145, w3: 2.8919741415078186, bias:1.049795241108232, loss: 0.5160377948291908
Epoch: 697, w1: -2.3046569820180562, w2: -0.54386983009668

Epoch: 780, w1: -2.302239069222636, w2: -0.6312549820701936, w3: 3.046331368332676, bias:1.0710076640945612, loss: 0.5152713576113234
Epoch: 781, w1: -2.302212351859764, w2: -0.6322292359110678, w3: 3.048114683594668, bias:1.071235927157484, loss: 0.5152629810187618
Epoch: 782, w1: -2.302185687973056, w2: -0.6332017234150691, w3: 3.0498967591377006, bias:1.0714636540616422, loss: 0.5152546206709864
Epoch: 783, w1: -2.3021590774154608, w2: -0.6341724481112461, w3: 3.0516775962217726, bias:1.0716908459539523, loss: 0.5152462765243638
Epoch: 784, w1: -2.3021325200398834, w2: -0.6351414135213187, w3: 3.0534571961051418, bias:1.0719175039785165, loss: 0.5152379485354049
Epoch: 785, w1: -2.302106015699196, w2: -0.6361086231596921, w3: 3.055235560044326, bias:1.0721436292766366, loss: 0.5152296366607637
Epoch: 786, w1: -2.3020795642462457, w2: -0.6370740805334701, w3: 3.057012689294109, bias:1.0723692229868258, loss: 0.5152213408572411
Epoch: 787, w1: -2.302053165533866, w2: -0.63803778914247

Epoch: 864, w1: -2.3001673706170194, w2: -0.7072574037235813, w3: 3.191924973903383, bias:1.088416917250302, loss: 0.5146204588514608
Epoch: 865, w1: -2.300144642621804, w2: -0.7080949159597635, w3: 3.1936083845466494, bias:1.0886038944656502, loss: 0.514613307519598
Epoch: 866, w1: -2.3001219562474238, w2: -0.7089309340690622, w3: 3.1952906555013847, bias:1.0887904230755787, loss: 0.5146061692241122
Epoch: 867, w1: -2.300099311364139, w2: -0.7097654610129217, w3: 3.196971787890686, bias:1.0889765040258543, loss: 0.514599043931831
Epoch: 868, w1: -2.300076707842555, w2: -0.7105984997466018, w3: 3.1986517828361287, bias:1.0891621382601855, loss: 0.5145919316096919
Epoch: 869, w1: -2.300054145553624, w2: -0.7114300532191908, w3: 3.200330641457766, bias:1.0893473267202283, loss: 0.514584832224735
Epoch: 870, w1: -2.300031624368645, w2: -0.7122601243736182, w3: 3.2020083648741333, bias:1.0895320703455922, loss: 0.5145777457441076
Epoch: 871, w1: -2.3000091441592656, w2: -0.7130887161466677

Epoch: 954, w1: -2.298274215139203, w2: -0.7769821934643751, w3: 3.3389956294302445, bias:1.1035543003648596, loss: 0.5140254080491485
Epoch: 955, w1: -2.2982547556237263, w2: -0.777696400634044, w3: 3.3405807929981206, bias:1.103704463318958, loss: 0.514019308694048
Epoch: 956, w1: -2.298235327496542, w2: -0.7784093569126567, w3: 3.3421649121221866, bias:1.1038542551816697, loss: 0.5140132197734947
Epoch: 957, w1: -2.2982159306603114, w2: -0.7791210647546689, w3: 3.3437479877974314, bias:1.1040036767344126, loss: 0.5140071412624923
Epoch: 958, w1: -2.298196565018043, w2: -0.7798315266094369, w3: 3.345330021017519, bias:1.1041527287569812, loss: 0.5140010731361184
Epoch: 959, w1: -2.298177230473089, w2: -0.7805407449212278, w3: 3.346911012774789, bias:1.1043014120275492, loss: 0.5139950153695308
Epoch: 960, w1: -2.2981579269291466, w2: -0.7812487221292314, w3: 3.348490964060261, bias:1.1044497273226737, loss: 0.5139889679379619
Epoch: 961, w1: -2.2981386542902555, w2: -0.78195546066757

In [25]:
scratch_model.predict(X_test)

248    0.229858
197    0.174532
133    0.729640
169    0.251350
736    0.701720
         ...   
422    0.193835
815    0.187315
466    0.187315
559    0.703583
6      0.199975
Length: 268, dtype: float64

### Create a neural network using tensorflow that has three input nodes, one output node, and that uses sigmoid activation function in the output node and binary cross-entropy loss function.


In [26]:
keras_model = keras.Sequential([keras.layers.Dense(1,
                                                  input_shape = (3, ),
                                                  activation = 'sigmoid',
                                                  kernel_initializer = 'ones',
                                                  bias_initializer = 'zeros')])

In [27]:
keras_model.compile(optimizer ='adam',
                   loss='binary_crossentropy',
                   metrics = ['accuracy'])

keras_model.fit(X_train, y_train, epochs = 1000)

Epoch 1/1000
20/20 [==============================] - 0s 808us/step - loss: 1.0698 - accuracy: 0.4045
Epoch 2/1000
20/20 [==============================] - 0s 1ms/step - loss: 1.0537 - accuracy: 0.4029
Epoch 3/1000
20/20 [==============================] - 0s 1ms/step - loss: 1.0378 - accuracy: 0.3981
Epoch 4/1000
20/20 [==============================] - 0s 1ms/step - loss: 1.0223 - accuracy: 0.3965
Epoch 5/1000
20/20 [==============================] - 0s 1ms/step - loss: 1.0074 - accuracy: 0.3981
Epoch 6/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.9926 - accuracy: 0.3949
Epoch 7/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.9785 - accuracy: 0.3933
Epoch 8/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.9647 - accuracy: 0.3933
Epoch 9/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.9515 - accuracy: 0.3949
Epoch 10/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.9385 - accuracy: 0.39

20/20 [==============================] - 0s 786us/step - loss: 0.5696 - accuracy: 0.7095
Epoch 161/1000
20/20 [==============================] - 0s 828us/step - loss: 0.5688 - accuracy: 0.7079
Epoch 162/1000
20/20 [==============================] - 0s 998us/step - loss: 0.5681 - accuracy: 0.7079
Epoch 163/1000
20/20 [==============================] - 0s 792us/step - loss: 0.5674 - accuracy: 0.7095
Epoch 164/1000
20/20 [==============================] - 0s 821us/step - loss: 0.5668 - accuracy: 0.7111
Epoch 165/1000
20/20 [==============================] - 0s 740us/step - loss: 0.5661 - accuracy: 0.7175
Epoch 166/1000
20/20 [==============================] - 0s 797us/step - loss: 0.5654 - accuracy: 0.7191
Epoch 167/1000
20/20 [==============================] - 0s 804us/step - loss: 0.5647 - accuracy: 0.7159
Epoch 168/1000
20/20 [==============================] - 0s 864us/step - loss: 0.5641 - accuracy: 0.7255
Epoch 169/1000
20/20 [==============================] - 0s 761us/step - loss: 0

20/20 [==============================] - 0s 766us/step - loss: 0.5325 - accuracy: 0.7673
Epoch 240/1000
20/20 [==============================] - 0s 912us/step - loss: 0.5323 - accuracy: 0.7673
Epoch 241/1000
20/20 [==============================] - 0s 895us/step - loss: 0.5319 - accuracy: 0.7673
Epoch 242/1000
20/20 [==============================] - 0s 899us/step - loss: 0.5317 - accuracy: 0.7673
Epoch 243/1000
20/20 [==============================] - 0s 852us/step - loss: 0.5314 - accuracy: 0.7673
Epoch 244/1000
20/20 [==============================] - 0s 811us/step - loss: 0.5312 - accuracy: 0.7673
Epoch 245/1000
20/20 [==============================] - 0s 872us/step - loss: 0.5309 - accuracy: 0.7673
Epoch 246/1000
20/20 [==============================] - 0s 968us/step - loss: 0.5307 - accuracy: 0.7673
Epoch 247/1000
20/20 [==============================] - 0s 983us/step - loss: 0.5304 - accuracy: 0.7673
Epoch 248/1000
20/20 [==============================] - 0s 985us/step - loss: 0

20/20 [==============================] - 0s 995us/step - loss: 0.5149 - accuracy: 0.7673
Epoch 398/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5149 - accuracy: 0.7673
Epoch 399/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5148 - accuracy: 0.7673
Epoch 400/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5149 - accuracy: 0.7673
Epoch 401/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5147 - accuracy: 0.7673
Epoch 402/1000
20/20 [==============================] - 0s 958us/step - loss: 0.5147 - accuracy: 0.7673
Epoch 403/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5147 - accuracy: 0.7673
Epoch 404/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5146 - accuracy: 0.7673
Epoch 405/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5146 - accuracy: 0.7673
Epoch 406/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5146 - accuracy

20/20 [==============================] - 0s 1ms/step - loss: 0.5128 - accuracy: 0.7673
Epoch 478/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5128 - accuracy: 0.7673
Epoch 479/1000
20/20 [==============================] - 0s 980us/step - loss: 0.5127 - accuracy: 0.7673
Epoch 480/1000
20/20 [==============================] - 0s 927us/step - loss: 0.5128 - accuracy: 0.7673
Epoch 481/1000
20/20 [==============================] - 0s 1000us/step - loss: 0.5127 - accuracy: 0.7673
Epoch 482/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5127 - accuracy: 0.7673
Epoch 483/1000
20/20 [==============================] - 0s 943us/step - loss: 0.5127 - accuracy: 0.7673
Epoch 484/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5127 - accuracy: 0.7673
Epoch 485/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5127 - accuracy: 0.7673
Epoch 486/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5126 - acc

20/20 [==============================] - 0s 916us/step - loss: 0.5109 - accuracy: 0.7689
Epoch 636/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5109 - accuracy: 0.7689
Epoch 637/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5108 - accuracy: 0.7689
Epoch 638/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5108 - accuracy: 0.7689
Epoch 639/1000
20/20 [==============================] - 0s 915us/step - loss: 0.5108 - accuracy: 0.7689
Epoch 640/1000
20/20 [==============================] - 0s 960us/step - loss: 0.5108 - accuracy: 0.7689
Epoch 641/1000
20/20 [==============================] - 0s 977us/step - loss: 0.5108 - accuracy: 0.7689
Epoch 642/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5108 - accuracy: 0.7689
Epoch 643/1000
20/20 [==============================] - 0s 899us/step - loss: 0.5108 - accuracy: 0.7689
Epoch 644/1000
20/20 [==============================] - 0s 1ms/step - loss: 0.5108 - ac

20/20 [==============================] - 0s 958us/step - loss: 0.5100 - accuracy: 0.7689
Epoch 794/1000
20/20 [==============================] - 0s 998us/step - loss: 0.5100 - accuracy: 0.7689
Epoch 795/1000
20/20 [==============================] - 0s 897us/step - loss: 0.5100 - accuracy: 0.7689
Epoch 796/1000
20/20 [==============================] - 0s 888us/step - loss: 0.5100 - accuracy: 0.7689
Epoch 797/1000
20/20 [==============================] - 0s 935us/step - loss: 0.5100 - accuracy: 0.7689
Epoch 798/1000
20/20 [==============================] - 0s 852us/step - loss: 0.5100 - accuracy: 0.7689
Epoch 799/1000
20/20 [==============================] - 0s 837us/step - loss: 0.5099 - accuracy: 0.7689
Epoch 800/1000
20/20 [==============================] - 0s 856us/step - loss: 0.5100 - accuracy: 0.7689
Epoch 801/1000
20/20 [==============================] - 0s 917us/step - loss: 0.5100 - accuracy: 0.7689
Epoch 802/1000
20/20 [==============================] - 0s 939us/step - loss: 0

20/20 [==============================] - 0s 836us/step - loss: 0.5096 - accuracy: 0.7673
Epoch 952/1000
20/20 [==============================] - 0s 896us/step - loss: 0.5096 - accuracy: 0.7673
Epoch 953/1000
20/20 [==============================] - 0s 902us/step - loss: 0.5096 - accuracy: 0.7673
Epoch 954/1000
20/20 [==============================] - 0s 847us/step - loss: 0.5096 - accuracy: 0.7673
Epoch 955/1000
20/20 [==============================] - 0s 843us/step - loss: 0.5096 - accuracy: 0.7673
Epoch 956/1000
20/20 [==============================] - 0s 956us/step - loss: 0.5096 - accuracy: 0.7673
Epoch 957/1000
20/20 [==============================] - 0s 897us/step - loss: 0.5096 - accuracy: 0.7673
Epoch 958/1000
20/20 [==============================] - 0s 846us/step - loss: 0.5096 - accuracy: 0.7673
Epoch 959/1000
20/20 [==============================] - 0s 947us/step - loss: 0.5096 - accuracy: 0.7673
Epoch 960/1000
20/20 [==============================] - 0s 880us/step - loss: 0

### Compare performance of the tensorflow neural network to the performance of the neural network created from scratch.

In [28]:
y_test

248    1
197    0
133    1
169    0
736    0
      ..
422    0
815    0
466    0
559    1
6      0
Name: Survived, Length: 268, dtype: int64

In [29]:
keras_model.predict(X_test)

array([[0.24778786],
       [0.15645382],
       [0.72068334],
       [0.27816758],
       [0.69222707],
       [0.7853222 ],
       [0.17866379],
       [0.1706461 ],
       [0.15269005],
       [0.16349176],
       [0.19722456],
       [0.18897483],
       [0.16401318],
       [0.6910802 ],
       [0.19264486],
       [0.18336397],
       [0.19291589],
       [0.1797669 ],
       [0.19666916],
       [0.1992845 ],
       [0.19122547],
       [0.17951739],
       [0.18956336],
       [0.8318225 ],
       [0.67956156],
       [0.2817164 ],
       [0.19673571],
       [0.18781638],
       [0.19357109],
       [0.19701886],
       [0.19942799],
       [0.18979284],
       [0.24897113],
       [0.193281  ],
       [0.34690738],
       [0.25665596],
       [0.7173373 ],
       [0.7242721 ],
       [0.27103996],
       [0.93929446],
       [0.68688565],
       [0.17951739],
       [0.20929527],
       [0.17887488],
       [0.19169843],
       [0.18486214],
       [0.73634934],
       [0.145

In [30]:
scratch_model.predict(X_test)

248    0.229858
197    0.174532
133    0.729640
169    0.251350
736    0.701720
         ...   
422    0.193835
815    0.187315
466    0.187315
559    0.703583
6      0.199975
Length: 268, dtype: float64

In [31]:
keras_model.get_weights()

[array([[-2.2678945 ],
        [-0.98241484],
        [ 5.9493246 ]], dtype=float32),
 array([0.99875367], dtype=float32)]

In [32]:
scratch_model.fit(X_train, y_train, 1000, 0.51)

Epoch: 1, w1: 0.8094506077346826, w2: 0.9324956629116281, w3: 0.9955389566048864, bias:-0.1650672262140765, loss: 1.0773497796966287
Epoch: 2, w1: 0.6403108764555319, w2: 0.8756282463757742, w3: 0.9927967275763201, bias:-0.3007245524187672, loss: 0.9504540645956774
Epoch: 3, w1: 0.4928631193278467, w2: 0.8291678131154406, w3: 0.9917055667444425, bias:-0.407946431141632, loss: 0.8579108731605325
Epoch: 4, w1: 0.36549057254169287, w2: 0.7921213165461127, w3: 0.992090786506443, bias:-0.4897016911695306, loss: 0.7931755806386545
Epoch: 5, w1: 0.2555146311321695, w2: 0.7630967380451104, w3: 0.9937243594454235, bias:-0.5499311866632984, loss: 0.7488632561534064
Epoch: 6, w1: 0.16000459293138292, w2: 0.7406422952789964, w3: 0.996373370807918, bias:-0.5926444892431924, loss: 0.7185579691053476
Epoch: 7, w1: 0.07626131191747981, w2: 0.7234445966250262, w3: 0.999828111245624, bias:-0.6214165614250001, loss: 0.6974780739344998
Epoch: 8, w1: 0.002003776831434795, w2: 0.7104010241100751, w3: 1.0039

Epoch: 70, w1: -1.5928497837635327, w2: 0.6795692090663432, w3: 1.3139784185417331, bias:0.18322741508651627, loss: 0.5448781890972018
Epoch: 71, w1: -1.605179001866232, w2: 0.6781691754224876, w3: 1.3180599012038872, bias:0.19223495565785856, loss: 0.5443622635562184
Epoch: 72, w1: -1.6172875858036306, w2: 0.6767343816780549, w3: 1.3221201362342712, bias:0.20110155074437636, loss: 0.5438627649271252
Epoch: 73, w1: -1.629179880949528, w2: 0.6752656945451211, w3: 1.326159533971708, bias:0.20982997225476735, loss: 0.5433790895076885
Epoch: 74, w1: -1.640860138475984, w2: 0.6737639581861217, w3: 1.3301784942543409, bias:0.21842292753394954, loss: 0.5429106573170562
Epoch: 75, w1: -1.6523325174344405, w2: 0.6722299948722341, w3: 1.3341774067208525, bias:0.22688306099487834, loss: 0.5424569111184342
Epoch: 76, w1: -1.663601086795551, w2: 0.6706646056276047, w3: 1.3381566511040246, bias:0.23521295572705114, loss: 0.5420173154825585
Epoch: 77, w1: -1.6746698274493155, w2: 0.6690685708583335, 

Epoch: 135, w1: -2.0756029707779473, w2: 0.5430507338579793, w3: 1.5478076312184474, bias:0.5638098509952391, loss: 0.5296101520591495
Epoch: 136, w1: -2.079634940368704, w2: 0.5405336028329686, w3: 1.5510511918728913, bias:0.567450497560743, loss: 0.5295160764647621
Epoch: 137, w1: -2.0836006876773046, w2: 0.5380103683166577, w3: 1.5542871467664736, bias:0.5710468860946011, loss: 0.5294237785520127
Epoch: 138, w1: -2.0875013443652164, w2: 0.5354812264641972, w3: 1.5575155898524586, bias:0.5745997255257933, loss: 0.5293332042334743
Epoch: 139, w1: -2.0913380207793617, w2: 0.53294636954812, w3: 1.5607366132583504, bias:0.5781097117488716, loss: 0.5292443012418466
Epoch: 140, w1: -2.095111806408417, w2: 0.5304059860455727, w3: 1.5639503073282364, bias:0.5815775279022379, loss: 0.5291570190649444
Epoch: 141, w1: -2.098823770328061, w2: 0.527860260723167, w3: 1.5671567606639778, bias:0.5850038446394106, loss: 0.529071308883132
Epoch: 142, w1: -2.1024749616354628, w2: 0.5253093747195297, w3

Epoch: 197, w1: -2.233661351293584, w2: 0.38114800136558846, w3: 1.7372781110148765, bias:0.7268304452889577, loss: 0.5258021305297673
Epoch: 198, w1: -2.235119035804455, w2: 0.3785134568625142, w3: 1.7401766166828214, bias:0.7286945476238674, loss: 0.5257599980580513
Epoch: 199, w1: -2.2365529321939337, w2: 0.37587996779725164, w3: 1.7430710216685092, bias:0.7305413181154563, loss: 0.5257181959543024
Epoch: 200, w1: -2.2379634174248464, w2: 0.3732475944014446, w3: 1.7459613573707518, bias:0.7323710007836104, loss: 0.5256767168756755
Epoch: 201, w1: -2.2393508622271, w2: 0.3706163957850393, w3: 1.7488476546906586, bias:0.7341838357450934, loss: 0.5256355536980765
Epoch: 202, w1: -2.240715631209996, w2: 0.36798642995623704, w3: 1.7517299440409455, bias:0.7359800592820755, loss: 0.5255946995091983
Epoch: 203, w1: -2.2420580829722625, w2: 0.3653577538410427, w3: 1.754608255355043, bias:0.7377599039093022, loss: 0.5255541476017966
Epoch: 204, w1: -2.24337857020985, w2: 0.36273042330241756,

Epoch: 266, w1: -2.2938328658614715, w2: 0.20421092210687475, w3: 1.9289547663927633, bias:0.8248390045520975, loss: 0.5234171680684437
Epoch: 267, w1: -2.294283523158704, w2: 0.20174536940089902, w3: 1.9316241911937146, bias:0.8259221598444935, loss: 0.5233879250114682
Epoch: 268, w1: -2.2947261550341156, w2: 0.19928317628054318, w3: 1.934290872321006, bias:0.8269983518339848, loss: 0.5233587895869702
Epoch: 269, w1: -2.295160886202969, w2: 0.196824357325639, w3: 1.9369548213377759, bias:0.8280676647254415, loss: 0.5233297606918312
Epoch: 270, w1: -2.295587839454203, w2: 0.1943689267662065, w3: 1.939616049658385, bias:0.8291301814844363, loss: 0.5233008372487684
Epoch: 271, w1: -2.2960071356812133, w2: 0.19191689848809437, w3: 1.9422745685508367, bias:0.8301859838566673, loss: 0.5232720182055832
Epoch: 272, w1: -2.296418893912105, w2: 0.1894682860385272, w3: 1.9449303891391547, bias:0.8312351523870535, loss: 0.5232433025344275
Epoch: 273, w1: -2.296823231339429, w2: 0.1870231026315615

...

Epoch: 597, w1: -2.3081451459642355, w2: -0.41918725528510353, w3: 2.697714206998168, bias:1.0188470932056939, loss: 0.5171311532770267
Epoch: 598, w1: -2.308107257819746, w2: -0.4205523824848611, w3: 2.699747589666378, bias:1.019196733886918, loss: 0.5171188889590782
Epoch: 599, w1: -2.3080694217864886, w2: -0.421914961767829, w3: 2.701779468588907, bias:1.019545555566825, loss: 0.5171066519806642
Epoch: 600, w1: -2.3080316383403194, w2: -0.4232749981783311, w3: 2.703809845421858, bias:1.0198935604357597, loss: 0.5170944422599907
Epoch: 601, w1: -2.3079939079454346, w2: -0.4246324967523161, w3: 2.7058387218184388, bias:1.0202407506722118, loss: 0.5170822597155551
Epoch: 602, w1: -2.307956231054559, w2: -0.42598746251734004, w3: 2.707866099428977, bias:1.0205871284429473, loss: 0.5170701042661442
Epoch: 603, w1: -2.307918608109134, w2: -0.4273399004925486, w3: 2.709891979900935, bias:1.0209326959031422, loss: 0.5170579758308269
Epoch: 604, w1: -2.307881039539503, w2: -0.428689815688660

Epoch: 703, w1: -2.3044681303397674, w2: -0.5506421108098793, w3: 2.905190743077236, bias:1.051725789350646, loss: 0.5159688319944634
Epoch: 704, w1: -2.3044368782586804, w2: -0.5517635714680866, w3: 2.907073443207693, bias:1.0519990279729068, loss: 0.5159590611841268
Epoch: 705, w1: -2.3044056897065412, w2: -0.552882970972195, w3: 2.908954801113818, bias:1.052271632382859, loss: 0.5159493104584617
Epoch: 706, w1: -2.304374564588716, w2: -0.5540003134557168, w3: 2.910834818199762, bias:1.052543603997182, loss: 0.515939579760914
Epoch: 707, w1: -2.3043435028089334, w2: -0.5551156030438432, w3: 2.9127134958676564, bias:1.0528149442281463, loss: 0.5159298690351181
Epoch: 708, w1: -2.3043125042693227, w2: -0.5562288438534551, w3: 2.9145908355176178, bias:1.0530856544836444, loss: 0.5159201782249073
Epoch: 709, w1: -2.3042815688704485, w2: -0.5573400399931336, w3: 2.916466838547752, bias:1.0533557361672234, loss: 0.5159105072742987
Epoch: 710, w1: -2.3042506965113474, w2: -0.558449195563171

Epoch: 774, w1: -2.302400504626229, w2: -0.6253721673969745, w3: 3.035605371882423, bias:1.069626761797769, loss: 0.5153219607690209
Epoch: 775, w1: -2.302373463321978, w2: -0.6263570937812655, w3: 3.0373961520160617, bias:1.069858266057769, loss: 0.5153134857832777
Epoch: 776, w1: -2.3023464763747263, w2: -0.6273402324997113, w3: 3.039185684834005, bias:1.0700892272176716, loss: 0.5153050273071859
Epoch: 777, w1: -2.3023195436378963, w2: -0.6283215871256242, w3: 3.0409739716067636, bias:1.0703196464415563, loss: 0.5152965852962362
Epoch: 778, w1: -2.3022926649648037, w2: -0.6293011612249048, w3: 3.0427610136030903, bias:1.0705495248906078, loss: 0.5152881597060629
Epoch: 779, w1: -2.302265840208671, w2: -0.6302789583560552, w3: 3.0445468120899806, bias:1.0707788637231304, loss: 0.5152797504924486
Epoch: 780, w1: -2.302239069222636, w2: -0.6312549820701936, w3: 3.046331368332676, bias:1.0710076640945612, loss: 0.5152713576113234
Epoch: 781, w1: -2.302212351859764, w2: -0.63222923591106

Epoch: 858, w1: -2.3003046199343324, w2: -0.7022007870184831, w3: 3.1818005134986103, bias:1.0872855800204726, loss: 0.5146636424775489
Epoch: 859, w1: -2.3002816394704517, w2: -0.7030473265542877, w3: 3.183490785943724, bias:1.0874752688455236, loss: 0.5146564122248571
Epoch: 860, w1: -2.300258701413003, w2: -0.7038923540639324, w3: 3.185179911929597, bias:1.087664503346968, loss: 0.5146491952098153
Epoch: 861, w1: -2.300235805630209, w2: -0.704735872546244, w3: 3.1868678925885168, bias:1.0878532844830577, loss: 0.5146419913986083
Epoch: 862, w1: -2.300212951990627, w2: -0.7055778849937864, w3: 3.1885547290512335, bias:1.0880416132099475, loss: 0.5146348007575274
Epoch: 863, w1: -2.300190140363152, w2: -0.7064183943928732, w3: 3.1902404224469616, bias:1.088229490481702, loss: 0.5146276232529758
Epoch: 864, w1: -2.3001673706170194, w2: -0.7072574037235813, w3: 3.191924973903383, bias:1.088416917250302, loss: 0.5146204588514608
Epoch: 865, w1: -2.300144642621804, w2: -0.7080949159597635

Epoch: 936, w1: -2.298629966868455, w2: -0.763909733145463, w3: 3.3102829430395664, bias:1.1007870099233692, loss: 0.5141370097126683
Epoch: 937, w1: -2.2986099253206667, w2: -0.7646468819463905, w3: 3.3118870782630125, bias:1.1009439880199374, loss: 0.5141307181719256
Epoch: 938, w1: -2.298589916973154, w2: -0.7653827347912939, w3: 3.3134901509037675, bias:1.1011005806773881, loss: 0.5141244375291367
Epoch: 939, w1: -2.2985699417222403, w2: -0.766117294228336, w3: 3.31509216198104, bias:1.1012567887070477, loss: 0.5141181677578736
Epoch: 940, w1: -2.298549999464607, w2: -0.7668505628003773, w3: 3.3166931125126746, bias:1.1014126129185466, loss: 0.5141119088317903
Epoch: 941, w1: -2.2985300900972923, w2: -0.767582543044988, w3: 3.318293003515155, bias:1.1015680541198238, loss: 0.5141056607246208
Epoch: 942, w1: -2.29851021351769, w2: -0.7683132374944587, w3: 3.3198918360036065, bias:1.1017231131171308, loss: 0.5140994234101844
Epoch: 943, w1: -2.2984903696235506, w2: -0.769042648675812

- At 1000 epochs, keras model yields an accuracy at 76.73% and loss = 0.5095.
- At 1000 epochs, the model built from scratch has loss = 0.51
- The values of w1, w2, w3 are slightly different between the two models.